In [10]:
import sys
libpath = "/home/pablo/tesis/tsm-pipeline/code"
if libpath not in sys.path:
    sys.path.append(libpath)
    
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import docker
docker_client = docker.from_env()

# Configura todo lo necesario en el ambiente
from scripts.docker import run_tsm, ExperimentSettings, read_dbs_dataset
from scripts.evaluate import evaluate, EvaluationSettings
from os.path import join as path_join

defaults = {
    "bash_config_path": "/home/pablo/tesis/tsm-pipeline/code/scripts/config.sh",
}
defaults_evalute = {
    "cache_root":"/home/pablo/dbcache",    
}

In [12]:
tainted_path = read_dbs_dataset("/home/pablo/tesis/tsm-pipeline/experiments/tesis/tainted_path_top_100.txt")
len(tainted_path)

100

In [13]:
from sklearn.model_selection import train_test_split
from numpy.random import RandomState

# Using a fixed random state to have reproducible results
rand = RandomState(seed=5)

train, test = train_test_split(tainted_path, train_size=.5, shuffle=True, random_state=rand)
train[0:10]

['chunkai1312/shields/8a89dad5bd52689f23e5c613f9f97b51c52f7866',
 'RuffApps/Apps/90e13d23b0593aa45f35fdac736b8e19c6616a9d',
 'bahamas10/node-static-route/c89a2793ac8ae34565602a9523b187e39e50b710',
 'alecperkins/missive/1782326c3cfd6d1816e724a55bb985309d98411b',
 'brianleroux/lawnchair/4a20309b14cee3b6eb74b24c688c5a4a5a98f879',
 'bkw/node-dronestream/09fedd578dfb6fe7f62ca990becb5cffec9154df',
 'TevinLi/amWiki/8756a085fd234b70e153c53e93f499592d393958',
 'azproduction/lmd/03410c36839728de683c697670e1b205dc8193ee',
 'ably/ably-js/202d6515c32638aad12eaed78c4905d14b63a649',
 'GoogleChromeLabs/ui-element-samples/cb21b7cb6b4de707cdd1856becb018d9b28c455b']

Calculo un split de 50/50 con tainted path top 100

In [6]:
results_dir = "/home/pablo/results/bb_path_1"

In [ ]:
results_dir = '/home/pablo/results/seldonstar_nosql_1'

In [7]:
# training
train_settings = ExperimentSettings(
    name="benjamin_button_path_1",
    query_type="path",
    project_list=train,
    results_dir=results_dir,
    **defaults
)

run_tsm(docker_client, train_settings, block=True)

running at container 62d077c94a5fad6a9f2073452d967e6639b78f336311bf7a3cb4184230bd809f. Use `docker logs 62d077c94a5fad6a9f2073452d967e6639b78f336311bf7a3cb4184230bd809f --tail 10 --follow` to follow progress
g: /home/pablo/results/bb_path_1/*/TaintedPathWorse-*/reprScores.txt


In [8]:
!cat /home/pablo/results/bb_path_1/averaged-results.csv|wc -l

128


In [14]:
# evaluation
external_predicate_file = path_join(results_dir, 'averaged-results.csv')
worse_settings = EvaluationSettings(
    search_path="/home/pablo/tesis/tsm-pipeline/lib-worse/codeql/javascript/ql:/home/pablo/.codeql/packages/codeql/javascript-upgrades/0.0.3",
    cli_version="2.13.1",
    db_cli_version="2.5.2",
    query_file="/home/pablo/tesis/tsm-pipeline/tsm-atm-pipeline/src/tsm/evaluation/TaintedPathWorseTSM.ql",
    external_predicate_file=external_predicate_file,
    **defaults_evalute,
)

print("Evaluating worse with %d dbs" % (len(test)))
evaluate(
    settings=worse_settings,
    output_dir=path_join(results_dir, 'worse'),
    dbs=test,
)

Evaluating worse with 50 dbs


In [15]:
v0_settings = EvaluationSettings(
    search_path="/home/pablo/codeqlv0/javascript/ql/lib",
    cli_version="2.13.1",
    db_cli_version="2.5.2",
    query_file="/home/pablo/tesis/tsm-pipeline/tsm-evaluation/tsm-evaluation/src/PathEvaluation.ql",
    **defaults_evalute,
)

print("Evaluating v0 with %d dbs" % (len(test)))
evaluate(
    settings=v0_settings,
    output_dir=path_join(results_dir, 'v0'),
    dbs=test,
)

Evaluating v0 with 50 dbs


In [16]:
from scripts.calculate_scores import calculate_scores

calculate_scores(
    results_folder=results_dir,
    cleanup_base_dir="/tmp",
)

(0.0, 0.0, 0.0)